<a href="https://colab.research.google.com/github/SriVidyaYeluripati/SITTA/blob/Vidya/SITTA_implmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [10]:
from IPython import get_ipython
from IPython.display import display
import kagglehub
kagglehub.login()
yeluripatisrividya_plant_pathology_sitta_path = kagglehub.dataset_download('yeluripatisrividya/plant-pathology-sitta')

100%|██████████| 779M/779M [00:39<00:00, 20.8MB/s]

Extracting files...


Kaggle credentials set.
Kaggle credentials successfully validated.


In [11]:
print(yeluripatisrividya_plant_pathology_sitta_path)

/root/.cache/kagglehub/datasets/yeluripatisrividya/plant-pathology-sitta/versions/1


In [5]:
!ls -l

total 4
drwxr-xr-x 1 root root 4096 Mar  4 14:26 sample_data


In [1]:
!pip install torch torchvision numpy pyyaml opencv-python matplotlib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 14.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [2]:
!pip install torch-fidelity lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.2 MB/s eta 0:00:00


In [8]:
import torch
import torch.distributed as dist
import torch.multiprocessing as mp

def setup(rank, world_size):
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)


In [3]:
!git clone https://github.com/SriVidyaYeluripati/SITTA.git -b Vidya

Cloning into 'SITTA'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 56 (delta 26), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 33.33 KiB | 11.11 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [12]:
%cd /root/.cache/kagglehub/datasets/yeluripatisrividya/plant-pathology-sitta/versions/1/images

/root/.cache/kagglehub/datasets/yeluripatisrividya/plant-pathology-sitta/versions/1/images


# Step 2: Dataset Preparation

In [13]:
import os
import shutil
import random
import glob

# Define dataset paths
dataset_path = "/root/.cache/kagglehub/datasets/yeluripatisrividya/plant-pathology-sitta/versions/1/images"
output_dir = "/kaggle/working/dataset"

# Create required directories
os.makedirs(f"{output_dir}/leaves/trainA", exist_ok=True)
os.makedirs(f"{output_dir}/leaves/testA", exist_ok=True)
os.makedirs(f"{output_dir}/leaves/trainB", exist_ok=True)
os.makedirs(f"{output_dir}/leaves/testB", exist_ok=True)

# Get all image files
all_images = glob.glob(f"{dataset_path}/*.jpg")
random.shuffle(all_images)

# Split into 80% train, 20% test
split_idx = int(0.8 * len(all_images))
trainA_images = all_images[:split_idx]
testA_images = all_images[split_idx:]

# Copy images to trainA/testA
for img in trainA_images:
    shutil.copy(img, f"{output_dir}/leaves/trainA/")

for img in testA_images:
    shutil.copy(img, f"{output_dir}/leaves/testA/")

# Select 200 random images as "textures" for trainB
num_textures = min(200, len(trainA_images))
trainB_images = random.sample(trainA_images, num_textures)

# Move them to trainB (simulating textures)
for img in trainB_images:
    shutil.copy(img, f"{output_dir}/leaves/trainB/")

# Move 10% of trainB to testB
testB_size = int(0.1 * len(trainB_images))
testB_images = trainB_images[:testB_size]

for img in testB_images:
    shutil.copy(img, f"{output_dir}/leaves/testB/")

print("✅ Dataset structured successfully!")

✅ Dataset structured successfully!


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 3: Resize Images

In [14]:
from PIL import Image
import os
import glob

# Define input and output directories
dataset_dirs = ["trainA", "testA", "trainB", "testB"]
base_dir = "/kaggle/working/dataset/leaves"

# Target size
TARGET_SIZE = (288, 288)

for dataset in dataset_dirs:
    input_folder = os.path.join(base_dir, dataset)
    resized_folder = os.path.join(base_dir, f"{dataset}_resized")
    os.makedirs(resized_folder, exist_ok=True)

    # Resize all images
    for img_path in glob.glob(input_folder + "/*.jpg"):
        img = Image.open(img_path).convert("RGB")  # Convert to RGB format
        img = img.resize(TARGET_SIZE, Image.BICUBIC)  # Bicubic interpolation
        img.save(os.path.join(resized_folder, os.path.basename(img_path)))

    print(f"✅ Resized images saved in {resized_folder}")

print("🔥 All images resized successfully!")

✅ Resized images saved in /kaggle/working/dataset/leaves/trainA_resized
✅ Resized images saved in /kaggle/working/dataset/leaves/testA_resized
✅ Resized images saved in /kaggle/working/dataset/leaves/trainB_resized
✅ Resized images saved in /kaggle/working/dataset/leaves/testB_resized
🔥 All images resized successfully!


# Step 4: Update YAML

In [15]:
import yaml

# Load the YAML file
yaml_path = "/content/SITTA/configs/single2single.yaml"
with open(yaml_path, "r") as file:
    config = yaml.safe_load(file)

config["trainA_dir"] = "/kaggle/working/dataset/leaves/trainA_resized"
config["testA_dir"] = "/kaggle/working/dataset/leaves/testA_resized"
config["trainB_dir"] = "/kaggle/working/dataset/leaves/trainB_resized"
config["testB_dir"] = "/kaggle/working/dataset/leaves/testB_resized"

# Save updated YAML file
with open(yaml_path, "w") as file:
    yaml.dump(config, file)

print("✅ YAML updated successfully!")

✅ YAML updated successfully!


## DataLoaders

In [16]:
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

# Define dataset paths
dataset_path = "/kaggle/working/dataset/leaves"

# Define transformations
transform = transforms.Compose([
    transforms.Resize((288, 288)),
    transforms.ToTensor(),
])

# Custom Dataset Class
class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_paths = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith(('.jpg', '.png'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")  # Convert to RGB format
        if self.transform:
            image = self.transform(image)
        return image  # Return only the image (No label since it's unsupervised)

# Load datasets using CustomImageDataset
trainA_dataset = CustomImageDataset(image_dir=f"{dataset_path}/trainA_resized", transform=transform)
trainB_dataset = CustomImageDataset(image_dir=f"{dataset_path}/trainB_resized", transform=transform)

# Create DataLoaders
trainA_loader = DataLoader(trainA_dataset, batch_size=8, shuffle=True)
trainB_loader = DataLoader(trainB_dataset, batch_size=8, shuffle=True)

print(f"✅ Loaded {len(trainA_dataset)} images in TrainA and {len(trainB_dataset)} images in TrainB")


✅ Loaded 2913 images in TrainA and 200 images in TrainB


# Step 5: SITTA Model Implementation (With PONO)
This includes:

PONO Normalization Layer
SITTA Generator (with and without PONO)
SITTA Discriminator

## 5.1: Define PONO Normalization

In [17]:
import torch
import torch.nn as nn

# Define PONO Normalization Layer
class PONO(nn.Module):
    def __init__(self):
        super(PONO, self).__init__()

    def forward(self, x):
        mean = x.mean(dim=[2, 3], keepdim=True)
        std = x.std(dim=[2, 3], keepdim=True)
        return (x - mean) / (std + 1e-5), mean, std

##  5.2: Define SITTA Generator (With & Without PONO)

In [18]:
# Define SITTA Generator with option to enable/disable PONO
class SITTA_Generator(nn.Module):
    def __init__(self, use_pono=False):
        super(SITTA_Generator, self).__init__()
        self.use_pono = use_pono
        self.pono_layer = PONO() if use_pono else None

        # Encoder: Extract features
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 7, padding=3),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.ReLU()
        )

        # Decoder: Generate texture
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 3, 7, padding=3),
            nn.Tanh()
        )

    def forward(self, x):
        features = self.encoder(x)

        # Apply PONO if enabled
        if self.use_pono:
            features, _, _ = self.pono_layer(features)

        return self.decoder(features)

## 5.3: Define SITTA Discriminator

In [19]:
# Define SITTA Discriminator
class SITTA_Discriminator(nn.Module):
    def __init__(self):
        super(SITTA_Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 1, 4, stride=1, padding=1)
        )

    def forward(self, x):
        return self.model(x)

## 5.4: Initialize Models

In [20]:
# Initialize models
generator_without_pono = SITTA_Generator(use_pono=False).cuda()
generator_with_pono = SITTA_Generator(use_pono=True).cuda()
discriminator = SITTA_Discriminator().cuda()

print("✅ Generator (with & without PONO) and Discriminator initialized successfully!")

✅ Generator (with & without PONO) and Discriminator initialized successfully!


# Step 6: Training Function for SITTA (With & Without PONO)
✔ Includes:


* Adversarial Loss
* Cycle Consistency Loss
* Identity Loss
* Training Function
* Training Execution for both With & Without PONO

## 6.1: Define Loss Functions

In [21]:
import torch.optim as optim
import torch.nn.functional as F

# Define loss functions
adversarial_loss = nn.MSELoss()  # For GAN loss
cycle_loss = nn.L1Loss()         # Cycle consistency loss
identity_loss = nn.L1Loss()      # Identity loss

# Optimizers
optimizer_G = optim.Adam(generator_with_pono.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

print("✅ Loss functions and optimizers initialized!")

✅ Loss functions and optimizers initialized!


## 6.2: Define Training Function

In [22]:
import time

def train_sitta(generator, label, trainA_loader, trainB_loader, epochs=5):
    """
    Training function for SITTA generator.
    Runs a simple adversarial loss training loop.
    """
    print(f"🚀 Training {label} Generator...")

    for epoch in range(epochs):
        start_time = time.time()

        for real_A in trainA_loader:
            for real_B in trainB_loader:
                real_A, real_B = real_A.cuda(), real_B.cuda()
                fake_B = generator(real_A)

                # Compute loss
                adv_loss = adversarial_loss(discriminator(fake_B), torch.ones_like(discriminator(fake_B)))
                cyc_loss = cycle_loss(generator(fake_B), real_A)
                idt_loss = identity_loss(generator(real_B), real_B)
                gen_loss = adv_loss + 10 * cyc_loss + 5 * idt_loss

                # Optimize Generator
                optimizer_G.zero_grad()
                gen_loss.backward()
                optimizer_G.step()

        end_time = time.time() - start_time
        print(f"Epoch [{epoch+1}/{epochs}] - Gen Loss: {gen_loss.item():.4f} - Time: {end_time:.2f}s")

    print(f"✅ Training Complete for {label}!")

## 6.3: Train Both Models (With & Without PONO)

In [23]:
# Train without PONO
generator_without_pono = SITTA_Generator(use_pono=False).cuda()
train_sitta(generator_without_pono, "Without PONO", trainA_loader, trainB_loader)

# Train with PONO
generator_with_pono = SITTA_Generator(use_pono=True).cuda()
train_sitta(generator_with_pono, "With PONO", trainA_loader, trainB_loader)

print("✅ Training completed for both versions!")


🚀 Training Without PONO Generator...
Epoch [1/5] - Gen Loss: 7.8142 - Time: 874.57s
Epoch [2/5] - Gen Loss: 7.7085 - Time: 873.60s
Epoch [3/5] - Gen Loss: 8.7510 - Time: 873.59s
Epoch [4/5] - Gen Loss: 7.2198 - Time: 873.43s
Epoch [5/5] - Gen Loss: 7.9516 - Time: 873.53s
✅ Training Complete for Without PONO!
🚀 Training With PONO Generator...
Epoch [1/5] - Gen Loss: 7.2027 - Time: 909.75s
Epoch [2/5] - Gen Loss: 7.2146 - Time: 909.41s
Epoch [3/5] - Gen Loss: 6.2360 - Time: 909.23s
Epoch [4/5] - Gen Loss: 8.0040 - Time: 909.22s
Epoch [5/5] - Gen Loss: 6.9245 - Time: 909.32s
✅ Training Complete for With PONO!
✅ Training completed for both versions!


# Step 7: Evaluation Metrics for SITTA
✔ Includes:

* FID (Fréchet Inception Distance)
* LPIPS (Learned Perceptual Image Patch Similarity)
* VGG Loss
* Visualization of Results

## 7.1: Define FID Calculation

In [24]:
from torch_fidelity import calculate_metrics

def compute_fid(real_images, generated_images):
    """
    Compute FID between real and generated images.
    """
    metrics = calculate_metrics(
        input1=real_images,
        input2=generated_images,
        fid=True,
        cuda=torch.cuda.is_available()
    )
    return metrics["frechet_inception_distance"]

##  7.2: Define LPIPS Calculation

In [26]:
import lpips

lpips_model = lpips.LPIPS(net='alex')

def compute_lpips(real_images, generated_images):
    """
    Compute LPIPS score.
    """
    return lpips_model(real_images, generated_images).mean().item()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth


## 7.3: Update VGG Loss

In [27]:
import torchvision.models as models

class VGGLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.vgg = models.vgg19(pretrained=True).features[:16].eval()
        for param in self.vgg.parameters():
            param.requires_grad = False
        self.criterion = nn.L1Loss()

    def forward(self, generated, target):
        generated_features = self.vgg(generated)
        target_features = self.vgg(target).detach()
        return self.criterion(generated_features, target_features)

##  7.4: Evaluate Trained Models

In [28]:
print("🔍 Evaluating SITTA With & Without PONO...")

# Compute evaluation metrics
fid_without_pono = compute_fid(real_B, generator_without_pono(real_A))
lpips_without_pono = compute_lpips(real_B, generator_without_pono(real_A))

fid_with_pono = compute_fid(real_B, generator_with_pono(real_A))
lpips_with_pono = compute_lpips(real_B, generator_with_pono(real_A))

print(f"📊 FID Score Without PONO: {fid_without_pono:.2f}")
print(f"📊 LPIPS Score Without PONO: {lpips_without_pono:.3f}")
print(f"📊 FID Score With PONO: {fid_with_pono:.2f}")
print(f"📊 LPIPS Score With PONO: {lpips_with_pono:.3f}")


🔍 Evaluating SITTA With & Without PONO...


NameError: name 'real_B' is not defined

## 7.5: Visualizing Results

In [ ]:
import matplotlib.pyplot as plt

def visualize_results(real_A, fake_B):
    """
    Displays a side-by-side comparison of real images and their generated textures.
    """
    real_A = real_A.permute(0, 2, 3, 1).cpu().numpy()
    fake_B = fake_B.permute(0, 2, 3, 1).cpu().numpy()

    fig, axes = plt.subplots(2, len(real_A), figsize=(12, 5))

    for i in range(len(real_A)):
        axes[0, i].imshow(real_A[i])
        axes[0, i].axis("off")
        axes[1, i].imshow(fake_B[i])
        axes[1, i].axis("off")

    axes[0, 0].set_title("Original Shape")
    axes[1, 0].set_title("Generated Texture")
    plt.show()

# Display a batch of results
visualize_results(test_real_A, test_fake_B)

In [ ]:
import matplotlib.pyplot as plt

# Plot FID & LPIPS for PONO vs No PONO
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].bar(["Without PONO", "With PONO"], [fid_without_pono, fid_with_pono], color=['red', 'green'])
axes[0].set_title("FID Score Comparison")
axes[0].set_ylabel("FID Score")

axes[1].bar(["Without PONO", "With PONO"], [lpips_without_pono, lpips_with_pono], color=['red', 'green'])
axes[1].set_title("LPIPS Score Comparison")
axes[1].set_ylabel("LPIPS Score")

plt.show()
print("✅ PONO Ablation Study Completed!")


# Step 8: Compare SITTA With PONO vs Without PONO
🚀 This step will compare the model’s performance when using PONO vs not using PONO.
✔ We will train two versions of the model and evaluate them.

## 8.1: Define Training Function for PONO Comparison

## 8.2: Train and Evaluate PONO vs Non-PONO

# Step 9: Save and Export the Final Trained Model

## 9.1: Save Trained Models

In [ ]:
# Create directory for saved models
os.makedirs("/kaggle/working/sitta_models", exist_ok=True)

# Save trained models
torch.save(generator_without_pono.state_dict(), "/kaggle/working/sitta_models/generator_without_pono.pth")
torch.save(generator_with_pono.state_dict(), "/kaggle/working/sitta_models/generator_with_pono.pth")

print("✅ Models saved successfully!")


##  9.2: Reload and Test Saved Models

In [ ]:
# Load trained models for verification
loaded_gen_without_pono = SITTA_Generator(use_pono=False).cuda()
loaded_gen_without_pono.load_state_dict(torch.load("/kaggle/working/sitta_models/generator_without_pono.pth"))

loaded_gen_with_pono = SITTA_Generator(use_pono=True).cuda()
loaded_gen_with_pono.load_state_dict(torch.load("/kaggle/working/sitta_models/generator_with_pono.pth"))

print("✅ Models reloaded successfully!")

# Generate new images using the reloaded models
test_fake_B_without_pono = loaded_gen_without_pono(test_real_A).detach()
test_fake_B_with_pono = loaded_gen_with_pono(test_real_A).detach()

# Display reloaded model results
print("📊 Reloaded Model Results Comparison")
visualize_results(test_real_A, test_fake_B_without_pono)
visualize_results(test_real_A, test_fake_B_with_pono)


## 9.3: Clean Up Files to Save Space

In [ ]:
!rm -rf /kaggle/working/dataset  # Remove dataset if not needed anymore
!rm -rf /kaggle/working/1  # Remove copied repo if needed
print("✅ Cleaned up unnecessary files!")
